In [11]:
import json
import numpy as np
import pandas as pd
import os

from math import cos, sin, exp, sqrt, pi, e

from src.OspitalettoDataset import OspitalettoDataset
from src.NOAA2010Dataset import NOAA2010Dataset
from src.InsPireDataset import InsPireDataset

## Constants to be used across the notebook

In [12]:
load_or_write_new = int(input("Press 1 if you want to reload previous simulation values"))

if load_or_write_new == 1:
    simulation_values_path = os.path.join(".", "data", "simulation_values.json")
    
    with open(simulation_values_path, "r") as f:
        values = json.load(f)
    
    Tmin_i = values["Tmin_i"]
    Tmax_i = values["Tmax_i"]

    Tdhw = values["Tdhw"]

    Ts1 = values["Ts1"]
    Ts2 = values["Ts2"]

    DT_evap = values["DT_evap"]
    
else:
    # Min and Max temperatures to be received by the user.
    # Values [40,...,60]
    Tmin_i = float(input("Enter min setpoint temperature: "))
    Tmax_i = float(input("Enter max setpoint temperature: "))

    # Domestic hot water temperature.
    # Values [45,...,55]
    Tdhw = float(input("Enter domestic hot water temperature: "))

    # Our model supposes that the system has two different sources of heat.
    # Values [20,...,30]
    Ts1 = float(input("Temperature of source 1: "))
    Ts2 = float(input("Temperature of source 2: "))

    # difference in temperature between the supply (system->heat->house) and return (house -> heat -> system) pipes
    # Values [1,...,10]
    DT_evap = float(input("Temperature difference among the supply and return pipes: "))
    
    
print(f"* {Tmin_i=}\n"
      f"* {Tmax_i=}\n"
      f"* {Tdhw=}\n"
      f"* {Ts1=}\n"
      f"* {Ts2=}\n"
      f"* {DT_evap=}\n")

Press 1 if you want to reload previous simulation values 1


* Tmin_i=40.0
* Tmax_i=55.0
* Tdhw=48.0
* Ts1=23.0
* Ts2=25.0
* DT_evap=10.0



# Load Sources working hours

The file must be a boolean matrix $W$ of $24 x 7$. The rows represent the hours in a day, and the columns represent the days in the week. For all $i \in \{0,...,23\}$ and $j \in \{0,..., 6\}$ we have that $w_{i,j} \in \{0, 1\}$, where $w_{i,j} = 0$ means that the source doesn't produce energy at that hour $i$ on that day $j$. Similarly, $w_{i,j} = 1$ means that the source produces energy at that hour $i$ on that day $j$.

In [13]:
s1_schedule_path = os.path.join(".", "data", "private", "s1_source_fake_schedule.xlsx")
if load_or_write_new == 1:
    s1_schedule = np.array(values["s1_schedule"])
else:    
    s1_schedule = pd.read_excel(s1_schedule_path, index_col="Time").to_numpy()

print(f"* {s1_schedule_path=}\n"
      f"* {s1_schedule=}\n"
      f"* {s1_schedule[6,1]=}\n" # Tuesdays are closed at 6 (equal to 0)
      f"* {s1_schedule[7,1]=}\n") # but open are 7 (equal to 1)

* s1_schedule_path='./data/private/s1_source_fake_schedule.xlsx'
* s1_schedule=array([[0, 0, 1, 1, 1, 1, 0],
       [0, 0, 1, 1, 1, 1, 0],
       [0, 0, 1, 1, 1, 1, 0],
       [0, 0, 1, 1, 1, 1, 0],
       [0, 0, 1, 1, 1, 1, 0],
       [0, 0, 1, 1, 1, 1, 0],
       [0, 0, 1, 1, 1, 1, 0],
       [0, 1, 1, 1, 1, 1, 0],
       [0, 1, 1, 1, 1, 1, 0],
       [0, 1, 1, 1, 1, 1, 0],
       [0, 1, 1, 1, 1, 1, 0],
       [0, 1, 1, 1, 1, 1, 0],
       [0, 1, 1, 1, 1, 1, 0],
       [0, 1, 1, 1, 1, 1, 0],
       [0, 1, 1, 1, 1, 1, 0],
       [0, 1, 1, 1, 1, 1, 0],
       [0, 1, 1, 1, 1, 0, 0],
       [0, 1, 1, 1, 1, 0, 0],
       [0, 1, 1, 1, 1, 0, 0],
       [0, 1, 1, 1, 1, 0, 0],
       [0, 1, 1, 1, 1, 0, 0],
       [0, 1, 1, 1, 1, 0, 0],
       [0, 1, 1, 1, 1, 0, 0],
       [0, 1, 1, 1, 1, 0, 0]])
* s1_schedule[6,1]=0
* s1_schedule[7,1]=1



In [14]:
s2_schedule_path = os.path.join(".", "data", "private", "s2_source_fake_schedule.xlsx")
if load_or_write_new == 1:
    s2_schedule = np.array(values["s2_schedule"])
else: 
    s2_schedule = pd.read_excel(s2_schedule_path, index_col="Time").to_numpy()

print(f"* {s2_schedule_path=}\n"
      f"* {s2_schedule=}\n"
      f"* {s2_schedule[15,1]=}\n" # Tuesdays are closed at 15 (equal to 0)
      f"* {s2_schedule[16,1]=}\n") # but open are 16 (equal to 1)

* s2_schedule_path='./data/private/s2_source_fake_schedule.xlsx'
* s2_schedule=array([[1, 1, 0, 0, 1, 1, 1],
       [1, 1, 0, 0, 1, 1, 1],
       [1, 1, 0, 0, 1, 1, 1],
       [1, 1, 0, 0, 1, 1, 1],
       [1, 1, 0, 0, 1, 1, 1],
       [1, 1, 0, 0, 1, 1, 1],
       [1, 1, 0, 0, 1, 1, 1],
       [1, 1, 0, 1, 1, 1, 1],
       [1, 1, 0, 1, 1, 1, 1],
       [1, 1, 0, 1, 1, 1, 1],
       [1, 1, 0, 1, 1, 1, 1],
       [1, 1, 0, 1, 1, 1, 1],
       [1, 1, 0, 1, 1, 1, 1],
       [1, 1, 0, 1, 1, 1, 1],
       [1, 1, 0, 1, 1, 1, 1],
       [1, 1, 0, 1, 1, 1, 1],
       [1, 0, 0, 1, 1, 1, 1],
       [1, 0, 0, 1, 1, 1, 1],
       [1, 0, 0, 1, 1, 1, 1],
       [1, 0, 0, 1, 1, 1, 1],
       [1, 0, 0, 1, 1, 1, 1],
       [1, 0, 0, 1, 1, 1, 1],
       [1, 0, 0, 1, 1, 1, 1],
       [1, 0, 0, 1, 1, 1, 1]])
* s2_schedule[15,1]=1
* s2_schedule[16,1]=0



## Take note of the values for the current simulation

In [15]:
simulation_values = dict()
simulation_values["Tmin_i"] = Tmin_i
simulation_values["Tmax_i"] = Tmax_i
simulation_values["Tdhw"] = Tdhw
simulation_values["Ts1"] = Ts1
simulation_values["Ts2"] = Ts2
simulation_values["DT_evap"] = DT_evap
simulation_values["s1_schedule"] = s1_schedule.tolist()
simulation_values["s2_schedule"] = s2_schedule.tolist()

## Read the dataset

In [16]:
ospitaletto2019 = OspitalettoDataset()
noaa2010Dataset = NOAA2010Dataset()
insPireDataset = InsPireDataset()

AVAILABLE_DATASETS = dict()
AVAILABLE_DATASETS.update(ospitaletto2019.load_processed_data())
AVAILABLE_DATASETS.update(noaa2010Dataset.load_processed_data())
AVAILABLE_DATASETS.update(insPireDataset.load_processed_data())
AVAILABLE_DATASETS.keys()

dict_keys(['ospitaletto', 'miami_florida', 'fresno_california', 'olympia_washington', 'rochester_newyork', 'london_uk', 'madrid_spain', 'rome_italy', 'stuttgart_germany'])

In [71]:
dataset_to_work = "stuttgart_germany"
dataset_to_work

'stuttgart_germany'

In [72]:
Tamb_hourly = AVAILABLE_DATASETS[dataset_to_work]
Tamb_hourly

,air_temp,dayofyear,month,hourofyear,air_temp_fit,ground_temp,aquifer_temp,space_heating_temp,hot_water_temp,user_temp,source1_temp,source2_temp,net_temp,COP,E_loss_s,E_loss_r,E_loss_tot
timestamp,,,,,,,,,,,,,,,,,
2017-01-01 00:00:00,-1.60,1,1,1,0.954843,4.855297,9.952648,55.000000,48.0,54.860000,0.0,25.0,25.0,4.345285,0.040289,0.020289,0.060579
2017-01-01 01:00:00,3.65,1,1,2,0.953185,4.852557,9.952648,51.088295,48.0,51.026529,0.0,25.0,25.0,4.657865,0.040295,0.020295,0.060590
2017-01-01 02:00:00,8.75,1,1,3,0.951533,4.849819,9.952648,40.000000,48.0,40.160000,0.0,25.0,25.0,5.975779,0.040300,0.020300,0.060601
2017-01-01 03:00:00,8.45,1,1,4,0.949885,4.847085,9.952648,40.000000,48.0,40.160000,0.0,25.0,25.0,5.975779,0.040306,0.020306,0.060612
2017-01-01 04:00:00,8.25,1,1,5,0.948242,4.844353,9.952648,40.000000,48.0,40.160000,0.0,25.0,25.0,5.975779,0.040311,0.020311,0.060623
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2017-12-31 20:00:00,-1.45,365,12,8757,0.963198,4.866283,9.952648,55.000000,48.0,54.790000,0.0,25.0,25.0,4.350514,0.040267,0.020267,0.060535
2017-12-31 21:00:00,-1.50,365,12,8758,0.961517,4.863532,9.952648,55.000000,48.0,54.790000,0.0,25.0,25.0,4.350514,0.040273,0.020273,0.060546
2017-12-31 22:00:00,-1.50,365,12,8759,0.959842,4.860784,9.952648,55.000000,48.0,54.790000,0.0,25.0,25.0,4.350514,0.040278,0.020278,0.060557


## Space heating degree

The amount (in degrees) and for how long (in hours) of thermal heat required to keep the indoor building temperature at a comfortable level will vary depending on different climates. The base temperature selected was set to 17 °C.

The space heating hourly profile for each location is then retrieved with a time dependency according to the heating degree method, known as the "Integration method".

More info on calculation method: https://www.degreedays.net/introduction

In [73]:
def heating_degree(T_amb):
    T_base = 17
    return max(0, T_base - T_amb)

In [74]:
Tamb_hourly["heating_degree_days"] = Tamb_hourly.air_temp.apply(heating_degree)
Tamb_hourly["space_heating_dist"] = Tamb_hourly.heating_degree_days / sum(Tamb_hourly.heating_degree_days)
Tamb_hourly

,air_temp,dayofyear,month,hourofyear,air_temp_fit,ground_temp,aquifer_temp,space_heating_temp,hot_water_temp,user_temp,source1_temp,source2_temp,net_temp,COP,E_loss_s,E_loss_r,E_loss_tot,heating_degree_days,space_heating_dist
timestamp,,,,,,,,,,,,,,,,,,,
2017-01-01 00:00:00,-1.60,1,1,1,0.954843,4.855297,9.952648,55.000000,48.0,54.860000,0.0,25.0,25.0,4.345285,0.040289,0.020289,0.060579,18.60,0.000268
2017-01-01 01:00:00,3.65,1,1,2,0.953185,4.852557,9.952648,51.088295,48.0,51.026529,0.0,25.0,25.0,4.657865,0.040295,0.020295,0.060590,13.35,0.000193
2017-01-01 02:00:00,8.75,1,1,3,0.951533,4.849819,9.952648,40.000000,48.0,40.160000,0.0,25.0,25.0,5.975779,0.040300,0.020300,0.060601,8.25,0.000119
2017-01-01 03:00:00,8.45,1,1,4,0.949885,4.847085,9.952648,40.000000,48.0,40.160000,0.0,25.0,25.0,5.975779,0.040306,0.020306,0.060612,8.55,0.000123
2017-01-01 04:00:00,8.25,1,1,5,0.948242,4.844353,9.952648,40.000000,48.0,40.160000,0.0,25.0,25.0,5.975779,0.040311,0.020311,0.060623,8.75,0.000126
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2017-12-31 20:00:00,-1.45,365,12,8757,0.963198,4.866283,9.952648,55.000000,48.0,54.790000,0.0,25.0,25.0,4.350514,0.040267,0.020267,0.060535,18.45,0.000266
2017-12-31 21:00:00,-1.50,365,12,8758,0.961517,4.863532,9.952648,55.000000,48.0,54.790000,0.0,25.0,25.0,4.350514,0.040273,0.020273,0.060546,18.50,0.000267
2017-12-31 22:00:00,-1.50,365,12,8759,0.959842,4.860784,9.952648,55.000000,48.0,54.790000,0.0,25.0,25.0,4.350514,0.040278,0.020278,0.060557,18.50,0.000267


## Calculate Space Heating and Hot Water Temperature

The space Heating is calculated using a climatic curve. The Hot Water temperature is constant across the year.

In [75]:
def climatic_curve(Tamb_h):
    Tmin_o = 2.38 #minimum outdoor T threshold in which the space heating system turns on
    Tmax_o = 7.25 #maximum outdoor T threshold in which the space heating system turns off

    if Tamb_h <= Tmin_o:   
        Tsh = Tmax_i
    elif Tamb_h >= Tmax_o:
        Tsh = Tmin_i
    else:
        m = (Tmax_i-Tmin_i)/(Tmin_o-Tmax_o)
        b = -m*Tmin_o+Tmax_i
        Tsh = m*Tamb_h+b
    return Tsh
 

In [76]:
Tamb_hourly['space_heating_temp'] = Tamb_hourly.air_temp.apply(climatic_curve)
Tamb_hourly["hot_water_temp"] = Tdhw
Tamb_hourly

,air_temp,dayofyear,month,hourofyear,air_temp_fit,ground_temp,aquifer_temp,space_heating_temp,hot_water_temp,user_temp,source1_temp,source2_temp,net_temp,COP,E_loss_s,E_loss_r,E_loss_tot,heating_degree_days,space_heating_dist
timestamp,,,,,,,,,,,,,,,,,,,
2017-01-01 00:00:00,-1.60,1,1,1,0.954843,4.855297,9.952648,55.000000,48.0,54.860000,0.0,25.0,25.0,4.345285,0.040289,0.020289,0.060579,18.60,0.000268
2017-01-01 01:00:00,3.65,1,1,2,0.953185,4.852557,9.952648,51.088295,48.0,51.026529,0.0,25.0,25.0,4.657865,0.040295,0.020295,0.060590,13.35,0.000193
2017-01-01 02:00:00,8.75,1,1,3,0.951533,4.849819,9.952648,40.000000,48.0,40.160000,0.0,25.0,25.0,5.975779,0.040300,0.020300,0.060601,8.25,0.000119
2017-01-01 03:00:00,8.45,1,1,4,0.949885,4.847085,9.952648,40.000000,48.0,40.160000,0.0,25.0,25.0,5.975779,0.040306,0.020306,0.060612,8.55,0.000123
2017-01-01 04:00:00,8.25,1,1,5,0.948242,4.844353,9.952648,40.000000,48.0,40.160000,0.0,25.0,25.0,5.975779,0.040311,0.020311,0.060623,8.75,0.000126
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2017-12-31 20:00:00,-1.45,365,12,8757,0.963198,4.866283,9.952648,55.000000,48.0,54.790000,0.0,25.0,25.0,4.350514,0.040267,0.020267,0.060535,18.45,0.000266
2017-12-31 21:00:00,-1.50,365,12,8758,0.961517,4.863532,9.952648,55.000000,48.0,54.790000,0.0,25.0,25.0,4.350514,0.040273,0.020273,0.060546,18.50,0.000267
2017-12-31 22:00:00,-1.50,365,12,8759,0.959842,4.860784,9.952648,55.000000,48.0,54.790000,0.0,25.0,25.0,4.350514,0.040278,0.020278,0.060557,18.50,0.000267


## Calculate User Temperature

This is the temperature that the users receive by the system. The temperature is based on their personal preferences, and the current season of the year.

In [77]:
def Tuser(fila):
    Tsh = fila.space_heating_temp
    Tdhw = fila.hot_water_temp
    month = fila.month
    
    #Las proporciones dependen de la localización
    if month==1:
        resultado= 0.98*Tsh+0.02*Tdhw
       
    elif month==2:
        resultado=0.96*Tsh+0.04*Tdhw
    
    elif month==3:
        resultado=0.94*Tsh+0.06*Tdhw
    
    elif month==4:
        resultado=0.90*Tsh+0.1*Tdhw
    
    elif month==5:
        resultado=0.75*Tsh+0.25*Tdhw
    
    elif month==6:
        resultado=0.25*Tsh+0.75*Tdhw
    
    elif month==7:
        resultado=0*Tsh+1.0*Tdhw
    
    elif month==8:
        resultado=0*Tsh+1.0*Tdhw
    
    elif month==9:
        resultado=0.1*Tsh+0.9*Tdhw
    
    elif month==10:
        resultado=0.80*Tsh+0.2*Tdhw
    
    elif month==11:
        resultado=0.95*Tsh+0.05*Tdhw
    
    elif month==12:
        resultado=0.97*Tsh+0.03*Tdhw
        
    return resultado

In [78]:
Tamb_hourly["user_temp"] = Tamb_hourly.apply(Tuser, axis=1)
Tamb_hourly

,air_temp,dayofyear,month,hourofyear,air_temp_fit,ground_temp,aquifer_temp,space_heating_temp,hot_water_temp,user_temp,source1_temp,source2_temp,net_temp,COP,E_loss_s,E_loss_r,E_loss_tot,heating_degree_days,space_heating_dist
timestamp,,,,,,,,,,,,,,,,,,,
2017-01-01 00:00:00,-1.60,1,1,1,0.954843,4.855297,9.952648,55.000000,48.0,54.860000,0.0,25.0,25.0,4.345285,0.040289,0.020289,0.060579,18.60,0.000268
2017-01-01 01:00:00,3.65,1,1,2,0.953185,4.852557,9.952648,51.088295,48.0,51.026529,0.0,25.0,25.0,4.657865,0.040295,0.020295,0.060590,13.35,0.000193
2017-01-01 02:00:00,8.75,1,1,3,0.951533,4.849819,9.952648,40.000000,48.0,40.160000,0.0,25.0,25.0,5.975779,0.040300,0.020300,0.060601,8.25,0.000119
2017-01-01 03:00:00,8.45,1,1,4,0.949885,4.847085,9.952648,40.000000,48.0,40.160000,0.0,25.0,25.0,5.975779,0.040306,0.020306,0.060612,8.55,0.000123
2017-01-01 04:00:00,8.25,1,1,5,0.948242,4.844353,9.952648,40.000000,48.0,40.160000,0.0,25.0,25.0,5.975779,0.040311,0.020311,0.060623,8.75,0.000126
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2017-12-31 20:00:00,-1.45,365,12,8757,0.963198,4.866283,9.952648,55.000000,48.0,54.790000,0.0,25.0,25.0,4.350514,0.040267,0.020267,0.060535,18.45,0.000266
2017-12-31 21:00:00,-1.50,365,12,8758,0.961517,4.863532,9.952648,55.000000,48.0,54.790000,0.0,25.0,25.0,4.350514,0.040273,0.020273,0.060546,18.50,0.000267
2017-12-31 22:00:00,-1.50,365,12,8759,0.959842,4.860784,9.952648,55.000000,48.0,54.790000,0.0,25.0,25.0,4.350514,0.040278,0.020278,0.060557,18.50,0.000267


## Calculate Sources Temperature based on Working Hours

In [79]:
dayofweeks = Tamb_hourly.index.dayofweek
hours = Tamb_hourly.index.hour

In [80]:
Tamb_hourly["source1_temp"] = Ts1 * s1_schedule[hours, dayofweeks]
Tamb_hourly["source2_temp"] = Ts2 * s2_schedule[hours, dayofweeks]
Tamb_hourly

,air_temp,dayofyear,month,hourofyear,air_temp_fit,ground_temp,aquifer_temp,space_heating_temp,hot_water_temp,user_temp,source1_temp,source2_temp,net_temp,COP,E_loss_s,E_loss_r,E_loss_tot,heating_degree_days,space_heating_dist
timestamp,,,,,,,,,,,,,,,,,,,
2017-01-01 00:00:00,-1.60,1,1,1,0.954843,4.855297,9.952648,55.000000,48.0,54.860000,0.0,25.0,25.0,4.345285,0.040289,0.020289,0.060579,18.60,0.000268
2017-01-01 01:00:00,3.65,1,1,2,0.953185,4.852557,9.952648,51.088295,48.0,51.026529,0.0,25.0,25.0,4.657865,0.040295,0.020295,0.060590,13.35,0.000193
2017-01-01 02:00:00,8.75,1,1,3,0.951533,4.849819,9.952648,40.000000,48.0,40.160000,0.0,25.0,25.0,5.975779,0.040300,0.020300,0.060601,8.25,0.000119
2017-01-01 03:00:00,8.45,1,1,4,0.949885,4.847085,9.952648,40.000000,48.0,40.160000,0.0,25.0,25.0,5.975779,0.040306,0.020306,0.060612,8.55,0.000123
2017-01-01 04:00:00,8.25,1,1,5,0.948242,4.844353,9.952648,40.000000,48.0,40.160000,0.0,25.0,25.0,5.975779,0.040311,0.020311,0.060623,8.75,0.000126
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2017-12-31 20:00:00,-1.45,365,12,8757,0.963198,4.866283,9.952648,55.000000,48.0,54.790000,0.0,25.0,25.0,4.350514,0.040267,0.020267,0.060535,18.45,0.000266
2017-12-31 21:00:00,-1.50,365,12,8758,0.961517,4.863532,9.952648,55.000000,48.0,54.790000,0.0,25.0,25.0,4.350514,0.040273,0.020273,0.060546,18.50,0.000267
2017-12-31 22:00:00,-1.50,365,12,8759,0.959842,4.860784,9.952648,55.000000,48.0,54.790000,0.0,25.0,25.0,4.350514,0.040278,0.020278,0.060557,18.50,0.000267


## Calculate Network temperature

In [81]:
def calculate_tnet(temp_s1, temp_s2, temp_aq):
    if temp_s1 == 0.0 and temp_s2 == 0.0:
        return temp_aq
    elif temp_s1 == 0.0:
        return temp_s2
    elif temp_s2 == 0.0:
        return temp_s1
    else:
        return np.mean([temp_s1, temp_s2])

In [82]:
Tamb_hourly["net_temp"] = Tamb_hourly.apply(lambda fila: calculate_tnet(fila.source1_temp, fila.source2_temp, fila.aquifer_temp), axis=1)
Tamb_hourly

,air_temp,dayofyear,month,hourofyear,air_temp_fit,ground_temp,aquifer_temp,space_heating_temp,hot_water_temp,user_temp,source1_temp,source2_temp,net_temp,COP,E_loss_s,E_loss_r,E_loss_tot,heating_degree_days,space_heating_dist
timestamp,,,,,,,,,,,,,,,,,,,
2017-01-01 00:00:00,-1.60,1,1,1,0.954843,4.855297,9.952648,55.000000,48.0,54.860000,0.0,25.0,25.0,4.345285,0.040289,0.020289,0.060579,18.60,0.000268
2017-01-01 01:00:00,3.65,1,1,2,0.953185,4.852557,9.952648,51.088295,48.0,51.026529,0.0,25.0,25.0,4.657865,0.040295,0.020295,0.060590,13.35,0.000193
2017-01-01 02:00:00,8.75,1,1,3,0.951533,4.849819,9.952648,40.000000,48.0,40.160000,0.0,25.0,25.0,5.975779,0.040300,0.020300,0.060601,8.25,0.000119
2017-01-01 03:00:00,8.45,1,1,4,0.949885,4.847085,9.952648,40.000000,48.0,40.160000,0.0,25.0,25.0,5.975779,0.040306,0.020306,0.060612,8.55,0.000123
2017-01-01 04:00:00,8.25,1,1,5,0.948242,4.844353,9.952648,40.000000,48.0,40.160000,0.0,25.0,25.0,5.975779,0.040311,0.020311,0.060623,8.75,0.000126
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2017-12-31 20:00:00,-1.45,365,12,8757,0.963198,4.866283,9.952648,55.000000,48.0,54.790000,0.0,25.0,25.0,4.350514,0.040267,0.020267,0.060535,18.45,0.000266
2017-12-31 21:00:00,-1.50,365,12,8758,0.961517,4.863532,9.952648,55.000000,48.0,54.790000,0.0,25.0,25.0,4.350514,0.040273,0.020273,0.060546,18.50,0.000267
2017-12-31 22:00:00,-1.50,365,12,8759,0.959842,4.860784,9.952648,55.000000,48.0,54.790000,0.0,25.0,25.0,4.350514,0.040278,0.020278,0.060557,18.50,0.000267


# Coefficient of performance (COP)

In [83]:
DT_hx = 2.5
n_HP = 0.53
Te_o = Tamb_hourly.net_temp - DT_evap - DT_hx
Tc_o = Tamb_hourly.user_temp

Tamb_hourly['COP']= 1-n_HP + n_HP * (Tc_o + 273.15) / (Tc_o + DT_hx - Te_o)
Tamb_hourly

,air_temp,dayofyear,month,hourofyear,air_temp_fit,ground_temp,aquifer_temp,space_heating_temp,hot_water_temp,user_temp,source1_temp,source2_temp,net_temp,COP,E_loss_s,E_loss_r,E_loss_tot,heating_degree_days,space_heating_dist
timestamp,,,,,,,,,,,,,,,,,,,
2017-01-01 00:00:00,-1.60,1,1,1,0.954843,4.855297,9.952648,55.000000,48.0,54.860000,0.0,25.0,25.0,4.345285,0.040289,0.020289,0.060579,18.60,0.000268
2017-01-01 01:00:00,3.65,1,1,2,0.953185,4.852557,9.952648,51.088295,48.0,51.026529,0.0,25.0,25.0,4.657865,0.040295,0.020295,0.060590,13.35,0.000193
2017-01-01 02:00:00,8.75,1,1,3,0.951533,4.849819,9.952648,40.000000,48.0,40.160000,0.0,25.0,25.0,5.975779,0.040300,0.020300,0.060601,8.25,0.000119
2017-01-01 03:00:00,8.45,1,1,4,0.949885,4.847085,9.952648,40.000000,48.0,40.160000,0.0,25.0,25.0,5.975779,0.040306,0.020306,0.060612,8.55,0.000123
2017-01-01 04:00:00,8.25,1,1,5,0.948242,4.844353,9.952648,40.000000,48.0,40.160000,0.0,25.0,25.0,5.975779,0.040311,0.020311,0.060623,8.75,0.000126
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2017-12-31 20:00:00,-1.45,365,12,8757,0.963198,4.866283,9.952648,55.000000,48.0,54.790000,0.0,25.0,25.0,4.350514,0.040267,0.020267,0.060535,18.45,0.000266
2017-12-31 21:00:00,-1.50,365,12,8758,0.961517,4.863532,9.952648,55.000000,48.0,54.790000,0.0,25.0,25.0,4.350514,0.040273,0.020273,0.060546,18.50,0.000267
2017-12-31 22:00:00,-1.50,365,12,8759,0.959842,4.860784,9.952648,55.000000,48.0,54.790000,0.0,25.0,25.0,4.350514,0.040278,0.020278,0.060557,18.50,0.000267


# Heat losses

In [84]:
def heat_losses(T_net,T_gr, DT_evap):
    U = 2
    T_ret = T_net - DT_evap
    HL_s = (T_net - T_gr) * U / 1000 # Heat losses supply pipe [MWh]
    HL_r = (T_ret - T_gr) * U / 1000 # Heat losses return pipe [MWh]
    
    return pd.Series([HL_s, HL_r, HL_s + HL_r] , index=['E_loss_s','E_loss_r','E_loss_tot'])


In [85]:
x = Tamb_hourly.apply(lambda fila: heat_losses(fila["net_temp"], fila["ground_temp"], DT_evap), axis=1, result_type='expand')

In [86]:
Tamb_hourly['E_loss_s'] = x['E_loss_s']
Tamb_hourly['E_loss_r'] = x['E_loss_r']
Tamb_hourly['E_loss_tot'] = x['E_loss_tot']
Tamb_hourly

,air_temp,dayofyear,month,hourofyear,air_temp_fit,ground_temp,aquifer_temp,space_heating_temp,hot_water_temp,user_temp,source1_temp,source2_temp,net_temp,COP,E_loss_s,E_loss_r,E_loss_tot,heating_degree_days,space_heating_dist
timestamp,,,,,,,,,,,,,,,,,,,
2017-01-01 00:00:00,-1.60,1,1,1,0.954843,4.855297,9.952648,55.000000,48.0,54.860000,0.0,25.0,25.0,4.345285,0.040289,0.020289,0.060579,18.60,0.000268
2017-01-01 01:00:00,3.65,1,1,2,0.953185,4.852557,9.952648,51.088295,48.0,51.026529,0.0,25.0,25.0,4.657865,0.040295,0.020295,0.060590,13.35,0.000193
2017-01-01 02:00:00,8.75,1,1,3,0.951533,4.849819,9.952648,40.000000,48.0,40.160000,0.0,25.0,25.0,5.975779,0.040300,0.020300,0.060601,8.25,0.000119
2017-01-01 03:00:00,8.45,1,1,4,0.949885,4.847085,9.952648,40.000000,48.0,40.160000,0.0,25.0,25.0,5.975779,0.040306,0.020306,0.060612,8.55,0.000123
2017-01-01 04:00:00,8.25,1,1,5,0.948242,4.844353,9.952648,40.000000,48.0,40.160000,0.0,25.0,25.0,5.975779,0.040311,0.020311,0.060623,8.75,0.000126
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2017-12-31 20:00:00,-1.45,365,12,8757,0.963198,4.866283,9.952648,55.000000,48.0,54.790000,0.0,25.0,25.0,4.350514,0.040267,0.020267,0.060535,18.45,0.000266
2017-12-31 21:00:00,-1.50,365,12,8758,0.961517,4.863532,9.952648,55.000000,48.0,54.790000,0.0,25.0,25.0,4.350514,0.040273,0.020273,0.060546,18.50,0.000267
2017-12-31 22:00:00,-1.50,365,12,8759,0.959842,4.860784,9.952648,55.000000,48.0,54.790000,0.0,25.0,25.0,4.350514,0.040278,0.020278,0.060557,18.50,0.000267


## Save CSV version of the dataset and simulation values

In [87]:
AVAILABLE_DATASET_PATHS = {OspitalettoDataset.OSPITALETTO: ospitaletto2019.processed_dataset_path,
                           NOAA2010Dataset.MIAMI_FL: noaa2010Dataset.processed_miami_fl_dataset_path, 
                           NOAA2010Dataset.FRESNO_CA: noaa2010Dataset.processed_fresno_ca_dataset_path, 
                           NOAA2010Dataset.OLYMPIA_WA: noaa2010Dataset.processed_olympia_wa_dataset_path,
                           NOAA2010Dataset.ROCHESTER_NY: noaa2010Dataset.processed_rochester_ny_dataset_path,
                           InsPireDataset.LONDON_UK: insPireDataset.processed_london_uk_dataset_path,
                           InsPireDataset.MADRID_SPA: insPireDataset.processed_madrid_spa_dataset_path,
                           InsPireDataset.ROME_IT: insPireDataset.processed_rome_it_dataset_path,
                           InsPireDataset.STUTTGART_GER: insPireDataset.processed_stuttgart_ger_dataset_path,}

Tamb_hourly.to_csv(path_or_buf=AVAILABLE_DATASET_PATHS[dataset_to_work])

In [88]:
simulation_values_path = os.path.join(".", "data", "simulation_values.json")

with open(simulation_values_path, "w") as f:
    json.dump(simulation_values, f, sort_keys=True, indent=2)